<a href="https://colab.research.google.com/github/shahaansshah/3253-083_Group9/blob/main/3253_083_Group_9_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ontario Housing Data Preprocessing

This is a rough template to start off with. Add or edit sections & code as needed.

In [59]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

### Add Proximity to lake feature to housing data?

### Import Ontario Housing data

In [60]:
# Bart - you loaded the data the best, since you already had the elements in 'features' flattened into the dataframe! Please include your method

In [61]:
# Mark
# #load dataset
# from google.colab import files
# data = files.upload()
# df = pd.DataFrame(data)

# pd.options.display.max_columns = None   # to list all columns when displaying output
# #ontario_housing_data

In [62]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ontario_housing_data = pd.read_json('/content/drive/My Drive/Colab Notebooks/ontario_housing_cleaned.json', lines=True)

Mounted at /content/drive


In [63]:
# Shahaan (from Mark)
data = []
with open('/content/drive/My Drive/Colab Notebooks/ontario_housing_cleaned.json', 'r') as f:  # old version of data (no lake proximity)
  for line in f:
    try:
      json_obj = json.loads(line.strip())
      data.append(json_obj)
    except json.JSONDecodeError:
      continue

# Convert to df
ontario_housing_data = pd.json_normalize(data)

In [64]:
ontario_housing_data.head()

,version,data.address,data.subPremise,data.fullAddress,data.division,data.city,data.cityCode,data.closePrice,data.closeDate,data.daysOnMovoto,...,data.isPriceUp,data.priceChangeRaw,data.priceChange,data.priceChangeFriendlyPrice,data.lastListPriceRaw,data.lastListPrice,data.pricePerAcre,data.pricePerAcreRaw,data.pricePerAcreIntRaw,data
0,2.1.0,2480 Prince Michael Dr S #210,#210,"2480 Prince Michael Dr S #210 Oakville, ON L6H...",Halton Region,Oakville,2506.0,NaN,NaN,14 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.1.0,1176 Grange Rd,,"1176 Grange Rd Oakville, ON L6H 1P6",Halton Region,Oakville,2506.0,NaN,NaN,Just Listed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.1.0,3511 Post Road,,"3511 Post Road Oakville, ON L6H 7W5",,Oakville,NaN,NaN,NaN,6 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.1.0,95 Dundas St W #513,#513,"95 Dundas St W #513 Oakville, ON L6M 5N4",Halton Region,Oakville,2506.0,NaN,NaN,34 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.1.0,3137 William Rose Way,,"3137 William Rose Way Oakville, ON L6H 0T1",Halton Region,Oakville,2506.0,NaN,NaN,76 days,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Handle Missing or Mostly Missing Data

In [65]:
# look up which fields contain a lot of missing data
missing_values = [col for col in ontario_housing_data.columns if ontario_housing_data[col].isnull().any()]
print('Number of columns with missing values:', len(missing_values), '\n')

for col in missing_values:
  print(col, round(ontario_housing_data[col].isnull().mean(), 3)*100, '% missing values')

Number of columns with missing values: 138 

data.address 0.2 % missing values
data.subPremise 0.2 % missing values
data.fullAddress 0.2 % missing values
data.division 0.2 % missing values
data.city 0.2 % missing values
data.cityCode 13.8 % missing values
data.closePrice 100.0 % missing values
data.closeDate 100.0 % missing values
data.daysOnMovoto 0.2 % missing values
data.daysOnMovotoRaw 0.2 % missing values
data.description 0.2 % missing values
data.features 0.2 % missing values
data.id 0.2 % missing values
data.tnImgPath 0.2 % missing values
data.imagesURL 0.2 % missing values
data.photos 0.2 % missing values
data.latitude 0.2 % missing values
data.listDate 38.6 % missing values
data.listingCoAgent 0.3 % missing values
data.listingAgent 0.3 % missing values
data.listingAgentLicense 0.2 % missing values
data.listingOfficePhone 0.2 % missing values
data.listPrice 0.2 % missing values
data.longitude 0.2 % missing values
data.lotSize 0.2 % missing values
data.lotSizeRaw 95.199999999999

In [72]:
# Drop the 7 empty rows (all of those '0.2% missing values' entries) (I just knew it's 7 rows from prior dataset work - Shahaan)

# get their indices
[print(col, ontario_housing_data[ontario_housing_data[col].isnull()].index) if ontario_housing_data[col].isnull().sum() == 7 else None for col in ontario_housing_data.columns]

# display the rows
# dropped_rows = ontario_housing_data.iloc[[844, 1503, 1553, 1881, 1890, 1925, 1942]]
# dropped_rows

# drop the rows
try:
  ontario_housing_data = ontario_housing_data.drop(index=[844, 1503, 1553, 1881, 1890, 1925, 1942])
except Exception:
  pass

In [67]:
# recalculate number of missing_values
missing_values = [col for col in ontario_housing_data.columns if ontario_housing_data[col].isnull().any()]
print('Number of columns with missing values:', len(missing_values), '\n')

Number of columns with missing values: 42 



In [70]:
# get columns which are >50% empty
too_many_missing = [col for col in ontario_housing_data.columns if ontario_housing_data[col].isnull().mean() > 0.5]
print('Number of columns with more than 50% missing values:', len(too_many_missing), '\n')

# drop those columns
try:
  ontario_housing_data = ontario_housing_data.drop(columns=too_many_missing)
except Exception:
  pass

Number of columns with more than 50% missing values: 29 



In [71]:
# rerun missing_values
missing_values = [col for col in ontario_housing_data.columns if ontario_housing_data[col].isnull().any()]
print('Number of columns with missing values:', len(missing_values), '\n')

for col in missing_values:
  print(col, round(ontario_housing_data[col].isnull().mean(), 3)*100, '% missing values')

Number of columns with missing values: 13 

data.cityCode 13.600000000000001 % missing values
data.listDate 38.5 % missing values
data.listingCoAgent 0.1 % missing values
data.listingAgent 0.1 % missing values
data.sqftTotalRaw 3.8 % missing values
data.parking 0.5 % missing values
data.neighborhoodNGeoId 7.7 % missing values
data.thumbnail 1.7999999999999998 % missing values
data.photoCount1 1.7999999999999998 % missing values
data.garage 49.5 % missing values
data.fsa 0.0 % missing values
data.petiteImagePath 1.7999999999999998 % missing values
data.propertyTypeNameUrl 5.3 % missing values


In [73]:
# notice that 'data.garage' is almost half empty, but 'data.parking' is almost full - check if they hold the same kind of data (they do)
# print(ontario_housing_data['data.garage'].value_counts(), '\n', ontario_housing_data['data.parking'].value_counts()

# drop 'data.garage'
try:
  ontario_housing_data = ontario_housing_data.drop(columns=['data.garage'])
except Exception:
  pass

data.garage
1.0     882
2.0     488
0.0     200
3.0      28
4.0       5
26.0      3
11.0      2
32.0      1
38.0      1
Name: count, dtype: int64
data.parking
1.0      968
2.0      577
4.0      420
0.0      415
3.0      279
6.0      212
5.0      132
8.0       51
10.0      37
7.0       29
9.0       17
12.0      12
11.0       7
13.0       5
14.0       4
20.0       2
15.0       1
18.0       1
23.0       1
44.0       1
16.0       1
102.0      1
Name: count, dtype: int64


### Drop other unneccessary features

In [80]:
pd.options.display.max_columns = None
ontario_housing_data

,version,data.address,data.subPremise,data.fullAddress,data.division,data.city,data.cityCode,data.daysOnMovoto,data.daysOnMovotoRaw,data.description,data.features,data.id,data.tnImgPath,data.imagesURL,data.photos,data.latitude,data.listDate,data.listingCoAgent,data.listingAgent,data.listingAgentLicense,data.listingOfficePhone,data.listPrice,data.longitude,data.lotSize,data.sqftTotal,data.sqftTotalRaw,data.mlsDbNumber,data.mlsSysid,data.mls.createdAt,data.mls.disclaimer,data.mls.id,data.mls.mlsName,data.mls.showAVM,data.mls.showMovotoAgentSection,data.mls.showPriceHistExtendedFlag,data.mls.showPriceHistFlag,data.mls.showPrivateNotesFlag,data.mls.showPublicRecordSoldHistory,data.mls.showSoldDppFlag,data.mls.type,data.mlsNumber,data.neighborhoodName,data.neighborhoodN,data.numBathrooms,data.numBathroomsRaw,data.numBedrooms,data.numBedroomsRaw,data.officeColistName,data.officeListName,data.officeListPhone,data.parking,data.photoCount,data.priceRaw,data.price,data.priceSeo,data.propertyType,data.propertyTypeName,data.propertyTypeValue,data.propertyTypeDisplayName,data.state,data.status,data.yearBuilt,data.zipCode,data.pageUrlWithoutDomain,data.houseRealStatus,data.listingOfficeDescription,data.totalMonthlyFee,data.hoafee,data.hoafeeRaw,data.dppInactiveOnActive,data.dppInactive,data.priceChangedDate,data.updatedTime,data.hiddenByComplianceRule,data.propertyId,data.neighborhoodNGeoId,data.visibility,data.permitAvm,data.modificationTimestamp,data.createdAt,data.imageDownloaderStatus,data.onMarketDateTime,data.thumbnail,data.photoCount1,data.garage,data.fsa,data.isVOWListing,data.isHotHome,data.dppurl,data.isSold,data.listingByMovoto,data.isPriceReduced,data.label,data.labelclass,data.labelDisplayName,data.listingPriceFormat,data.addressRaw,data.address2,data.lotSizeUnit,data.sqftTotalUnit,data.comparableHomes,data.listDateLLFormat,data.listDateFormat,data.listDateUTC,data.pricePerSqft,data.pricePerSqftRaw,data.pricePerSqftIntRaw,data.isFavorite,data.petiteImagePath,data.propertyTypeNameUrl,status.code,status.msg
0,2.1.0,2480 Prince Michael Dr S #210,#210,"2480 Prince Michael Dr S #210 Oakville, ON L6H...",Halton Region,Oakville,2506.0,14 days,14.0,This Spectacular Condo Apartment Is Ready To W...,"[{'name': 'Amenities Utilities', 'value': [{'...",00a82f5f-ff8f-48b8-85c1-5f174407f661,https:mls-photos.ojo.cacan_crea_idx80d27036183...,[https:mls-photos.ojo.cacan_crea_idx80d2703618...,"[{'hashValue': None, 'imageFormat': 'jpeg', 'i...",43.497160,2024-06-12 00:00:00,,KASIA GORZKOWSKI,,,995000.0,-79.707940,,,NaN,1007.0,27036183,2021-03-30 05:15:12,,1007.0,CREA,True,True,YES,YES,True,True,YES,RETS,W8436376,...,Iroquois Ridge North,2,2.0,2,2.0,,REMAX EXPERTS,,2.0,27.0,995000.0,"$995,000","995,000",Single Family,SINGLE_FAMILY_HOUSE,1,Single Family,ON,Active,...,L6H 0H1,oakville-on2480-prince-michael-dr-s-210-oakvil...,ACTIVE,,854.0,$854month,854.0,False,False,2024-06-14 00:00:00,2024-07-01 19:01:29,False,89581c36-bf1f-4cc7-9f68-47556432dddc,8155.0,NO_RESTRICTION,True,2024-06-14 07:09:46,2024-06-19 00:59:29,0.0,2024-06-12 00:00:00,https:mls-photos.ojo.cacan_crea_idx80d27036183...,27.0,NaN,L6H,False,False,https:www.houseful.caoakville-on2480-prince-mi...,False,False,False,Active,active,For Sale,"$995,000",2480 Prince Michael Dr S #210,2480 Prince Michael Drive S #210,Sqft,Sqft,[],"June 12,2024",06122024,2024-06-12T00:00:00Z,...,...,0.0,False,https:mls-photos.ojo.cacan_crea_idx80d27036183...,single-family,0,Success
1,2.1.0,1176 Grange Rd,,"1176 Grange Rd Oakville, ON L6H 1P6",Halton Region,Oakville,2506.0,Just Listed,0.0,"Lovely 3 bedroom, two bath bungalow on charmin...","[{'name': 'Amenities Utilities', 'value': [{'...",011e1776-d546-4ea1-b499-d0e9220851f3,https:mls-photos.ojo.cacan_crea_idxba127113060...,[https:mls-photos.ojo.cacan_crea_idxba12711306...,"[{'hashValue': None, 'imageFormat': 'jpeg', 'i...",43.479693,2024-07-02 00:00:00,DANIELLA QUATTROCIOCCHI,MICHELLE BAILEY,,,1190000.0,-79.682036,,,0.0,1007.0,27113060,2021-03-30 05:15:12,,1007.

### Display correlation heatmap of remaining features that we believe are of interest
- drop redundant features
- drop useless features

### Scale Features

### Obtain training and testing data.

In [ ]:
X, y = ontario_housing_data.drop(columns=['data.priceRaw']), ontario_housing_data['data.priceRaw']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)